In [1]:
## Mathieu VANDECASTEELE - September 2018
## This is the first chatbot i've created. It is not really efficient since I used only 581 samples which is very less compared to
## the dataset use to develop actual chatbots. However, the concept remains,just
## need a bigger and better dataset to increase the efficiency.
## To use it, launch the last script.
import os, sys
import re
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
## VARIABLES
BATCH_SIZE = 4
EPOCHS = 20
LSTM_NODES = 128
MAX_SENTENCE_LENGTH = 200
VOCABULARY_SIZE = 2000
VECTOR_DIM = 100

In [3]:
## Initialisation and pre-processing
input_texts = []
target_texts = []
target_texts_inputs = []



In [4]:
with open("human_text.txt", encoding="utf8") as textfile1, open("robot_text.txt", encoding="utf8") as textfile2:
    for human_text, robot_text in zip(textfile1, textfile2):
        human_text = human_text.strip()
        robot_text = robot_text.strip()

        human_text = re.sub(r'\W', ' ', str(human_text))
        robot_text = re.sub(r'\W', ' ', str(robot_text))

        human_text = re.sub(r'\s+', ' ', human_text, flags=re.I)
        robot_text = re.sub(r'\s+', ' ', robot_text, flags=re.I)

        input_text = human_text
        target_text = robot_text + ' <eos>'
        target_text_input = '<sos> ' + robot_text


        #WE CAN REMOVE THIS LINE IF WE WANT TO USE ALL THE DB
        if len(input_text.split()) > MAX_SENTENCE_LENGTH:
            break;
            
        input_texts.append(input_text)
        target_texts.append(target_text)
        target_texts_inputs.append(target_text_input)
    
print("num samples:", len(input_texts))


num samples: 581


In [5]:
## Tokenizing and padding
tokenizer_inputs = Tokenizer(num_words=VOCABULARY_SIZE )
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)
word2idx_inputs = tokenizer_inputs.word_index
max_len_input = max(len(s) for s in input_sequences)

In [6]:
tokenizer_outputs = Tokenizer(num_words=VOCABULARY_SIZE , filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) # inefficient, oh well
target_sequences =tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)
word2idx_outputs = tokenizer_outputs.word_index
num_words_output = len(word2idx_outputs) + 1
max_len_target = max(len(s) for s in target_sequences)

In [7]:
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

In [9]:
## Load Word Embedding
print('Loading Word Vectors')
embedding_dic = {}
with open("glove.6b.100d.txt", encoding= "utf8") as vector_list:
    for line in vector_list:
        dimensions = line.split()
        word = dimensions[0]
        vector = np.asarray(dimensions[1:], dtype='float32')
        embedding_dic[word] = vector

num_words = min(VOCABULARY_SIZE, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, VECTOR_DIM ))

for word, i in word2idx_inputs.items():
     if i < num_words:
        embedding_vector = embedding_dic.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Loading Word Vectors


In [12]:
## Creating Input Embedding Layer and Output One Hot Encoded Layer
embedding_layer = Embedding(
 num_words,
 VECTOR_DIM,
 weights=[embedding_matrix],
 input_length=max_len_input,
 # trainable=True
)

decoder_targets_one_hot = np.zeros(
 (
     len(input_texts),
     max_len_target,
     num_words_output
 ),
 dtype='float32'
)

for i, d in enumerate(decoder_targets):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [11]:
##Creating the Model

encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(
 LSTM_NODES,
 return_state=True,
)
encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [13]:
decoder_inputs_placeholder = Input(shape=(max_len_target,))
decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
decoder_lstm = LSTM(
 LSTM_NODES,
 return_sequences=True,
 return_state=True,
)
decoder_outputs, _, _ = decoder_lstm(
 decoder_inputs_x,
 initial_state=encoder_states
)


In [14]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
model = Model([encoder_inputs_placeholder,
decoder_inputs_placeholder], decoder_outputs)
model.compile(
 optimizer='rmsprop',
 loss='categorical_crossentropy',
 metrics=['accuracy']
 )

In [16]:
r = model.fit(
 [encoder_inputs, decoder_inputs], decoder_targets_one_hot,
 batch_size=BATCH_SIZE,
 epochs=EPOCHS,
 validation_split=0.2,
)

Train on 464 samples, validate on 117 samples
Epoch 1/20
464/464 [==============================] - 38s 83ms/step - loss: 1.5249 - acc: 0.8121 - val_loss: 0.9234 - val_acc: 0.8570
Epoch 2/20
464/464 [==============================] - 32s 69ms/step - loss: 1.0342 - acc: 0.8369 - val_loss: 0.9134 - val_acc: 0.8619
Epoch 3/20
464/464 [==============================] - 32s 70ms/step - loss: 1.0007 - acc: 0.8372 - val_loss: 0.9061 - val_acc: 0.8643
Epoch 4/20
464/464 [==============================] - 32s 69ms/step - loss: 0.9733 - acc: 0.8391 - val_loss: 0.8920 - val_acc: 0.8661
Epoch 5/20
464/464 [==============================] - 34s 72ms/step - loss: 0.9465 - acc: 0.8412 - val_loss: 0.8798 - val_acc: 0.8678
Epoch 6/20
464/464 [==============================] - 31s 66ms/step - loss: 0.9208 - acc: 0.8436 - val_loss: 0.8729 - val_acc: 0.8687
Epoch 7/20
464/464 [==============================] - 33s 71ms/step - loss: 0.8960 - acc: 0.8451 - val_loss: 0.8682 - val_acc: 0.8695
Epoch 8/20
464/4

In [17]:
## Making predictions

encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [18]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [19]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [20]:
decoder_outputs, h, c = decoder_lstm(
 decoder_inputs_single_x,
 initial_state=decoder_states_inputs
)

In [21]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
# The sampling model
# inputs: y(t-1), h(t-1), c(t-1)
# outputs: y(t), h(t), c(t)
decoder_model = Model(
 [decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [24]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [27]:
def generate_response(input_seq):
 # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
 # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
 # Populate the first character of target sequence with the start character.

 # NOTE: tokenizer lower-cases all words
    target_seq[0, 0] = word2idx_outputs['<sos>']
 # if we get this we break
    eos = word2idx_outputs['<eos>']
 # Create the translation
    output_sentence = []
    for _ in range(max_len_target):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
 # Get next word
        idx = np.argmax(output_tokens[0, 0, :])
 # End sentence of EOS
        if eos == idx:
            break
        word = ''
        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)
 # Update the decoder input
 # which is just the word just generated
        target_seq[0, 0] = idx
 # Update states
        states_value = [h, c]
 # states_value = [h] # gru
    return ' '.join(output_sentence)

In [ ]:
## SCRIPT TO USE THE CHATBOT

while True:
 # Do some test translations
    i = np.random.choice(len(input_texts))
    input_seq = encoder_inputs[i:i+1]
    translation = generate_response(input_seq)
    print('-')
    print('Input:', input_texts[i])
    print('Response:', translation)
    ans = input("Continue? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break

-
Input: good news 
Response: hi
-
Input: can you send me some jazz songs you like 
Response: i m sure i m fine and you
-
Input: no recién supe de ti en la red social diaspora 
Response: ok ok
-
Input: i don t know you man 
Response: yes i m fine but you can call to talk to the bit
-
Input: cool what is your name 
Response: i m a bit of and you can t call
-
Input: yeah we had to cook lots of foot for a party 
Response: you like a bit
-
Input: i m going to have lunch now
Response: good
-
Input: you ve been off
Response: hi there i m a bit
-
Input: hey i was at dentist had an operation surgery
Response: hi a surgery of
-
Input: wow nice to meet you i go to bed
Response: good good night
-
Input: how long will you be charged 
Response: i m a bit of of and
-
Input: almost midnight what about your 
Response: 
-
Input: what the score for film lalaland on rottentomatoes 
Response: the bit is the world
-
Input: i hardcoded the timezote to eest
Response: is nice
-
Input: no i mean you can like s